# coremltools
Apple 에서 제공하는 coremltools의 convert를 이용해 pytorch 모델을 mlmodel로 변형가능하다. 이를 사용하기위해 설치해준다.

In [1]:
!pip install coremltools

# Pytorch 버전 선택
mlmodel convert 과정에서 `ValueError: unexpected number of inputs for node input.2 (_convolution): 13` 에러가 발생했다.
https://github.com/apple/coremltools/issues/1012
위 글을 참고하여 Pytorch 버전을 1.6.0으로 변경해준다. 
https://stackoverflow.com/questions/61273244/how-to-downgrade-torch-version-for-google-colab
버전 변경방법은 위 글을 참조했다.

In [2]:
!pip install torch==1.6.0 torchvision==0.7.0

In [5]:
import torch
import torchvision
import coremltools as ct

In [6]:
print(torch.__version__)

1.6.0


# mobilenet
Pytorch에서 제공하는 mobilenet을 바로 불러왔다.

In [7]:
torch_model = torchvision.models.mobilenet_v2()

In [8]:
print(torch_model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

# Tracing
Pytorch모델을 mlmodel로 변환하기 위해선 Script를 얻어내기위해 Tracing해주는 과정이 필요하다.

In [9]:
torch_model.eval()
example_input = torch.rand(1, 3, 256, 256)
traced_model = torch.jit.trace(torch_model, example_input)

In [10]:
mlmodel = ct.convert(traced_model, inputs=[ct.TensorType(shape=example_input.shape)])

Translating MIL ==> MLModel Ops: 100%|██████████| 706/706 [00:00<00:00, 1176.60 ops/s]


In [11]:
print(mlmodel)

input {
  name: "input.1"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 256
      shape: 256
      dataType: FLOAT32
    }
  }
}
output {
  name: "653"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==1.6.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.0"
  }
}



In [13]:
mlmodel.save("mobilenet.mlmodel")

# 다운로드
mlmodel로 convert가 완료되었다. 

In [16]:
from google.colab import files
files.download("mobilenet.mlmodel") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>